In [ ]:
!pip3 install requests

In [ ]:
import requests 
import random
import string
import json
from datetime import datetime

# base_url = "https://du-backend-t437.onrender.com/"
base_url = "http://localhost:8002"

ir_ids = [
    "IM7216", "IM6599", "IM1579", "IM9252", "IM2004", "IM4230", "IM7990", "IM0262",
    "IM1781", "IM1332", "IM9002", "IM4546", "IM4927", "IM6392", "IM2779", "IM9215",
    "IM8947", "IM8361", "IM2499", "IM0424", "IM6678", "IM9375", "IM8657", "IM5852", "IM1530"
]

In [ ]:

def createIr(new_id):  # Make sure this is defined
    path = "/api/add_ir_id"
    end_point = f"{base_url}{path}"
    payload = {"ir_id": new_id}

    try:
        response = requests.post(end_point, json=payload, timeout=10)
        response.raise_for_status()
        print(response.json())
    except requests.exceptions.Timeout:
        print("Request timed out.")
    except requests.exceptions.ConnectionError:
        print("Failed to connect to the server.")
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err.response.text}")
    except Exception as e:
        print(f"An error occurred: {e}")


for ir_id in ir_ids:
    createIr(ir_id)


In [ ]:
access_distribution = (
    [1] +                  # Admin (1 user)
    [2] * 5 +              # LDC (5 users)
    [3] * 8 +              # LS (8 users)
    [4] * (len(ir_ids) - (1 + 5 + 8))  # GC (Remaining users)
)

def register_ir(ir_id, access_level):
    path = "/api/register_new_ir"
    end_point = f"{base_url}{path}"
    data = {
        "ir_id": ir_id,
        "ir_name": f"Test User {ir_id}",
        "ir_email": f"{ir_id.lower()}@example.com",
        "ir_password": "TestPass123",
        "ir_access_level": access_level
    }
    
    try:
        response = requests.post(
            end_point,
            json=data,
            timeout=10
        )
        
        if response.ok:
            print(f"✅ Registered {ir_id} (Access: {access_level})")
        else:
            try:
                print(f"❌ Error {ir_id} (Access: {access_level}) → {response.status_code} | {response.json()}")
            except ValueError:
                print(f"❌ Error {ir_id} (Access: {access_level}) → {response.status_code} | {response.text}")
    
    except requests.exceptions.RequestException as e:
        print(f"🚫 Failed {ir_id}: {e}")

def populate_test_irs():
    for ir_id, access_level in zip(ir_ids, access_distribution):
        register_ir(ir_id, access_level)


populate_test_irs()

In [ ]:
def getAllRegisteredIr():
    path = "/api/irs"
    end_point = f"{base_url}{path}"
    response = requests.get(end_point)
    
    if response.ok:
        try:
            print("Result:", response.json())
        except ValueError:
            print("Result (non-JSON):", response.text)
    else:
        try:
            print("Error:", response.json())
        except ValueError:
            print("Error (non-JSON):", response.text)

getAllRegisteredIr()


In [ ]:
def createTeam():
    path = "/api/create_team"
    end_point = f"{base_url}{path}"
    data = {"name":"Relentless"}
    data = json.dumps(data)
    response = requests.post(end_point,data=data,headers={"Content-type":"application/json"})
    if response.ok:
        print("Success ", response.json())
    else:
        print("Failure ", response.json())

createTeam()

In [ ]:
def getAllTeams():
    path = "/api/teams"
    end_point = f"{base_url}{path}"
    response = requests.get(end_point)
    if response.ok:
        print("Success ", response.json())
    else:
        print("Failure ", response.json())

getAllTeams()

In [ ]:
def update_team_name(team_id, new_name):
    path = f"/api/update_team_name/{team_id}"  # <-- f-string here
    end_point = f"{base_url}{path}"
    payload = {"name": new_name}
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.patch(end_point, json=payload, headers=headers)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

update_team_name(3, "Dreame Relentless")

In [ ]:
def deleteTeam(team_id):
    path = f"/api/delete_team/{team_id}"
    end_point = f"{base_url}{path}"
    response = requests.delete(end_point)
    if response.ok:
        print("Success ", response.json())
    else:
        print("Failure ", response.json())

deleteTeam(2)

In [ ]:
def add_ir_to_team(ir_id, team_id, role):
    url = f"{base_url}/api/add_ir_to_team"
    payload = {
        "ir_id": ir_id,
        "team_id": team_id,
        "role": role
    }
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(url, json=payload, headers=headers, timeout=10)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        try:
            print(f"HTTP error occurred: {http_err} - {response.json()}")
        except Exception:
            print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

# Example usage:

add_ir_to_team("IM2499", 3, "IR")

In [ ]:
def get_team_members(team_id):
    url = f"{base_url}/api/team_members/{team_id}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        try:
            print(f"HTTP error occurred: {http_err} - {response.json()}")
        except Exception:
            print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

get_team_members(1) 

In [ ]:
def remove_ir_from_team(base_url, team_id, ir_id):
    url = f"{base_url}/api/remove_ir_from_team/{team_id}/{ir_id}"
    try:
        response = requests.delete(url, timeout=10)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        try:
            print(f"HTTP error occurred: {http_err} - {response.json()}")
        except Exception:
            print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

remove_ir_from_team(base_url, 1, "IM1332")

In [ ]:

def add_info_detail(base_url, ir_id, info_date, response, comments, info_name):
    url = f"{base_url}/api/add_info_detail/{ir_id}"
    payload = {
        "ir_id": ir_id,
        "info_date": info_date,  # Should be ISO format string, e.g. "2025-08-26T12:00:00+05:30"
        "response": response,    # Should be one of "A", "B", "C"
        "comments": comments,
        "info_name": info_name
    }
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(url, json=payload, headers=headers, timeout=10)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        try:
            print(f"HTTP error occurred: {http_err} - {response.json()}")
        except Exception:
            print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

# Example usage:
ir_id = "IM7216"
info_date = datetime.now().isoformat()  # Or use your preferred datetime string
response_option = "B"                   # Must be "A", "B", or "C"
comments = "Info went well"
info_name = "Vinay"

add_info_detail(base_url, ir_id, info_date, response_option, comments, info_name)

In [ ]:
def getAllInfoData(ir_id):
    path = f"/api/info_details/{ir_id}"
    end_point = f"{base_url}{path}"
    try:
        response = requests.get(end_point, timeout=10)
        response.raise_for_status()
        try:
            print("Result:", response.json())
        except ValueError:
            print("Result (non-JSON):", response.text)
    except requests.exceptions.HTTPError as http_err:
        try:
            print(f"HTTP error occurred: {http_err} - {response.json()}")
        except Exception:
            print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

getAllInfoData("IM7216")

In [ ]:
def update_ir_info_data(info_id):
    path = f"/api/update_info_detail/{info_id}"
    end_point = f"{base_url}{path}"
    payload = {"info_name":"Snehith","comments":"Updated via script","response":"A"}
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.put(end_point, json=payload, headers=headers)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

update_ir_info_data(3)

In [ ]:
def delete_info(info_id):
    path =f"/api/delete_info_detail/{info_id}"
    end_point = f"{base_url}{path}"
    try:
        response = requests.delete(end_point, timeout=10)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except Exception as e:
        print(f"An error occurred: {e}")

delete_info(3)